In [1]:
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from math import sqrt, tanh, cos, sin, pi
from numba import njit, prange

@njit(parallel=True)
def Bt(B0, k0, k1, k2, t):
    return B0*k0*tanh(l1*t)*cos((k2*t)**2)

@njit(parallel=True)
def Ct(C0, k0, k1, k2, t):
    return C0*k0*tanh(l1*t)*cos((k2*t)**2)

@njit(parallel=True)
def velocity_steady(A0, B0, C0, k0, k1, k2, x, y, z):
    
    u = A0*sqrt(3)*sin(z)+C0*cos(y)
    v = B0*sin(x)+A0*cos(z)
    w = C0*sin(y)+B0*cos(x)
    
    return u, v, w

@njit(parallel=True)
def velocity_usteady(A0, B0, C0, k0, k1, k2, x, y, z, t):

    Bt = Bt(B0, k0, k1, k2, t)
    Ct = Ct(C0, k0, k1, k2, t)
    
    u = A0*sqrt(3)*sin(z)+(C0+Ct)*cos(y)
    v = (B0+Bt)*sin(x)+A0*cos(z)
    w = (C0+Ct)*sin(y)+(B0+Bt)*cos(x)
    
    return u, v, w

@njit(parallel=True)
def ABC_steady(A0, B0, C0, k0, k1, k2, X, Y, Z):
    
    U = X.copy()*np.nan
    V = Y.copy()*np.nan
    W = Z.copy()*np.nan
    
    for i in prange(X.shape[0]):
        
        for j in prange(Y.shape[1]):
            
            for k in prange(Y.shape[2]):
                
                x = X[i, j, k]
                y = Y[i, j, k]
                z = Z[i, j, k]
                
                u, v, w = velocity_steady(A0, B0, C0, k0, k1, k2, x, y, z)
                
                U[i, j, k] = u
                V[i, j, k] = v
                W[i, j, k] = w
        
    return U, V, W
            
@njit(parallel=True)
def ABC_unsteady(A0, B0, C0, k0, k1, k2, X, Y, Z, t):
    
    for i in prange(Y.shape[0]):
        
        for j in prange(X.shape[1]):
            
            for k in prange(Z.shape[2]):
                
                for t in prange(T.shape[3]):
                
                    x = X[i, j, k]
                    y = Y[i, j, k]
                    z = Z[i, j, k]                
                
                    u, v, w = velocity(A0, B0, C0, k0, k1, k2, x, y, z, t)
                
                    U[i, j, k, t] = u
                    V[i, j, k, t] = v
                    W[i, j, k, t] = w

In [3]:
x = np.linspace(0, 2*pi, 100, endpoint = True)
y = x.copy()
z = x.copy()
X, Y, Z = np.meshgrid(x, x, x)

A0 = sqrt(3)
B0 = sqrt(2)
C0 = 1
k0 = .15
k1 = .05
k2 = .12

U, V, W = ABC_steady(A0, B0, C0, k0, k1, k2, X, Y, Z)
            
scipy.io.savemat('./ABC_steady.mat', {'u': U, 'v': V, 'w': W, 'x': x, 'y': y, 'z': z})

6.283185307179586
[0.         0.06346652 0.12693304 0.19039955 0.25386607 0.31733259
 0.38079911 0.44426563 0.50773215 0.57119866 0.63466518 0.6981317
 0.76159822 0.82506474 0.88853126 0.95199777 1.01546429 1.07893081
 1.14239733 1.20586385 1.26933037 1.33279688 1.3962634  1.45972992
 1.52319644 1.58666296 1.65012947 1.71359599 1.77706251 1.84052903
 1.90399555 1.96746207 2.03092858 2.0943951  2.15786162 2.22132814
 2.28479466 2.34826118 2.41172769 2.47519421 2.53866073 2.60212725
 2.66559377 2.72906028 2.7925268  2.85599332 2.91945984 2.98292636
 3.04639288 3.10985939 3.17332591 3.23679243 3.30025895 3.36372547
 3.42719199 3.4906585  3.55412502 3.61759154 3.68105806 3.74452458
 3.8079911  3.87145761 3.93492413 3.99839065 4.06185717 4.12532369
 4.1887902  4.25225672 4.31572324 4.37918976 4.44265628 4.5061228
 4.56958931 4.63305583 4.69652235 4.75998887 4.82345539 4.88692191
 4.95038842 5.01385494 5.07732146 5.14078798 5.2042545  5.26772102
 5.33118753 5.39465405 5.45812057 5.52158709 5